run the following scripts to generate the masks and a table of volumes (it has absolute and relative volumes) this is an example on how to generate the tables from freesurfer outputs

# Generate Combined Masks

The following code generates new masks based on segmentation data. If segmentation files do not exist for a given case, the case is added to the `no_seg` list.


In [ ]:
import os
import re
import pandas as pd
import glob
import nibabel as nib
import numpy as np
import glob
from fsl.wrappers import fslreorient2std

In [ ]:
# Define a dictionary (sumroi) that maps anatomical regions to lists of corresponding labels for ROIs.
# Each key in the dictionary represents a specific anatomical region, and the associated value is a list of labels
# identifying ROIs within that region.

sumroi = { 'Frontal': ['ctx.lh.caudalmiddlefrontal',
 'ctx.rh.caudalmiddlefrontal',
 'ctx.lh.lateralorbitofrontal',
 'ctx.rh.lateralorbitofrontal',
 'ctx.lh.medialorbitofrontal',
 'ctx.rh.medialorbitofrontal',
 'ctx.lh.parsopercularis',
 'ctx.rh.parsopercularis',
 'ctx.lh.parsorbitalis',
 'ctx.rh.parsorbitalis',
 'ctx.lh.parstriangularis',
 'ctx.rh.parstriangularis',
 'ctx.lh.precentral',
 'ctx.rh.precentral',
 'ctx.lh.rostralmiddlefrontal',
 'ctx.rh.rostralmiddlefrontal',
 'ctx.lh.superiorfrontal',
 'ctx.rh.superiorfrontal'],
      'Hippocampus' : ['Left.Hippocampus', 'Right.Hippocampus'],
     'Frontal_Parietal' : ['ctx.lh.paracentral', 'ctx.rh.paracentral'],
          'Parietal' : ['ctx.lh.inferiorparietal',
 'ctx.rh.inferiorparietal',
 'ctx.lh.postcentral',
 'ctx.rh.postcentral',
 'ctx.lh.precuneus',
 'ctx.rh.precuneus',
 'ctx.lh.superiorparietal',
 'ctx.rh.superiorparietal',
 'ctx.lh.supramarginal',
 'ctx.rh.supramarginal'],
          'Insula': ['ctx.lh.insula', 'ctx.rh.insula'],
          'Cingulate':['ctx.lh.caudalanteriorcingulate',
 'ctx.rh.caudalanteriorcingulate',
 'ctx.lh.isthmuscingulate',
 'ctx.rh.isthmuscingulate',
 'ctx.lh.posteriorcingulate',
 'ctx.rh.posteriorcingulate',
 'ctx.lh.rostralanteriorcingulate',
 'ctx.rh.rostralanteriorcingulate'],
          'Occipital': ['ctx.lh.cuneus',
 'ctx.rh.cuneus',
 'ctx.lh.lateraloccipital',
 'ctx.rh.lateraloccipital',
 'ctx.lh.lingual',
 'ctx.rh.lingual',
 'ctx.lh.pericalcarine',
 'ctx.rh.pericalcarine'],
          'BasalGanglia':['Left.Caudate',
 'Right.Caudate',
 'Left.Putamen',
 'Right.Putamen',
 'Left.Pallidum',
 'Right.Pallidum',
 'Left.Accumbens.area',
 'Right.Accumbens.area'],
          'Thalamus':['Left.Thalamus.Proper', 'Right.Thalamus.Proper'],
          'Cerebellum':['Left.Cerebellum.White.Matter',
 'Right.Cerebellum.White.Matter',
 'Left.Cerebellum.Cortex',
 'Right.Cerebellum.Cortex'],
          'CorpusCallosum':['CC_Posterior',
 'CC_Mid_Posterior',
 'CC_Central',
 'CC_Mid_Anterior',
 'CC_Anterior'],
          'Ventricles':['Left.Lateral.Ventricle',
 'Right.Lateral.Ventricle',
 'Left.Inf.Lat.Vent',
 'Right.Inf.Lat.Vent',
 'Left.choroid.plexus',
 'Right.choroid.plexus',
 '3rd.Ventricle',
 '4th.Ventricle',
 'CSF'],
        'VentralDC':['Left.VentralDC', 'Right.VentralDC'],
          'WM': ['Left.Cerebral.White.Matter', 'Right.Cerebral.White.Matter'],
          'Brainstem':['Brain.Stem'],
  'Temporal' : ['Left-Amygdala',
'Right-Amygdala',
'ctx-lh-entorhinal',
'ctx-lh-fusiform',
'ctx-lh-inferiortemporal',
'ctx-lh-middletemporal',
'ctx-lh-parahippocampal',
'ctx-lh-superiortemporal',
'ctx-lh-transversetemporal',
'ctx-rh-entorhinal',
'ctx-rh-fusiform',
'ctx-rh-inferiortemporal',
'ctx-rh-middletemporal',
'ctx-rh-parahippocampal',
'ctx-rh-superiortemporal',
'ctx-rh-transversetemporal',],
}
for i in sumroi:
    sumroi[i] += [i.replace('.','-') for i in sumroi[i]]
# Create a mapping from anatomical regions to numeric identifiers (our_map)
our_map = dict()
n = 1
for i in sumroi.keys():
    our_map[i] = n
    n += 1
our_map_inv = dict(zip(our_map.values(), our_map.keys()))


fs_labels_path = '/FreeSurferColorLUT.txt'# refer to freesurfer dir


d=dict()
d_inv = dict()
# Parse the FreeSurfer color lookup table and create mappings between label numbers and label names
with open(fs_labels_path,'r') as fin:
    for row in fin:
        #print(row)
        src = re.search(r'([0-9]{1,4}) *([^ ]*) *[0-9]{1,3} *[0-9]{1,3} *[0-9]{1,3} *[0-9].*', row)
        if src:
            if src[2]:
                #print(src[1], src[2])
                d[int(src[1])]=src[2]
                d_inv[src[2]]=int(src[1])
                
                
sumroi_num = dict()
for i in sumroi:
    sumroi_num[i] = []
    for j in sumroi[i]:
        sumroi_num[i].append(int(d_inv[j.replace('.','-')]))


In [ ]:
current_dir = !pwd
current_dir = current_dir[0]
main_dir = "/".join(current_dir.split("/")[:-1])
fs = glob.glob(f'{main_dir}/FREESURFER_SEG/*.reconall/mri/aparc.DKTatlas+aseg.mgz')
subs = glob.glob(f'{main_dir}FREESURFER_SEG/sub-*/mri')
dest_dirname = f'{main_dir}/QC_Freesurfer'

In [ ]:
#generate the new masks
no_seg =[]

for i in subs:
    #if the case that the segmentation does not exist
    try:
        # 1 Nifti conversion
        t1w_mgz = nib.load(os.path.join(i, 'T1.mgz'))
        t1w_nii = nib.Nifti1Image(t1w_mgz.get_fdata(), header = t1w_mgz.header, affine=t1w_mgz.affine)
        sid = i.split('/')[-2]
        dest_fullpath = os.path.join(dest_dirname, sid, 'T1.nii.gz')
        os.makedirs(os.path.dirname(dest_fullpath), exist_ok=True)
        nib.save(t1w_nii, dest_fullpath)

        # 2 Summary Masks
        mask_mgz = nib.load(os.path.join(i, 'aparc.DKTatlas+aseg.mgz'))
        mask_arr = mask_mgz.get_fdata().copy()

        for roi in our_map.keys():
            values = sumroi_num[roi]
            for v in values:
                mask_arr = np.where(mask_arr == v, our_map[roi]*1e4, mask_arr)



        mask_arr = np.where(mask_arr < 1e4, 0, mask_arr)       
        mask_arr /= 1e4


        mask_nii = nib.Nifti1Image(mask_arr, header= mask_mgz.header, affine=mask_mgz.affine)    
        nib.save(mask_nii, os.path.join(dest_dirname, sid, 'seg_mask.nii.gz'))    


        for v in np.unique(mask_arr).astype(int):
            if v != 0:
                roi_nii = nib.Nifti1Image(np.where(mask_arr==v, mask_arr,0), header= mask_mgz.header, affine=mask_mgz.affine)
                roi_nii_name = os.path.join(dest_dirname, sid, 'seg_{}.nii.gz'.format(our_map_inv[v]))
                nib.save(roi_nii, roi_nii_name)

            #rotation
        generierten_files = glob.glob(os.path.join(dest_dirname, sid, '*'))
        for i in generierten_files:
            fslreorient2std(i,i)

        
    except:
        print (f"{i} segmentation does not exist")
        no_seg.append(i)

print(no_seg)

### Calculating Volumes for Predefined ROIs

We can calculate the volumes of predefined regions of interest (ROIs) using the data from `fastsurfer_aseg.volume.stats.csv` and `freesurfer_aseg.volume.stats.csv`. The volumes can be calculated for individual ROIs or summarized for the left and right brain segments.


```python
import pandas as pd

# Load the data from fastsurfer_aseg.volume.stats.csv and freesurfer_aseg.volume.stats.csv
freesurfer_data = pd.read_csv('freesurfer_aseg.volume.stats.csv')
fastsurfer_data = pd.read_csv('fastsurfer_aseg.volume.stats.csv')



In [ ]:
#clean up the orignial table
df = freesurfer_data
df.rename(columns={"Measure:volume":"IDs"},inplace= True)
df.rename(columns={"EstimatedTotalIntraCranialVol":"eTIV"},inplace= True)
df.to_csv("freesurfer_vol_unsum.csv",index=False) # save the vols before any summation

In [ ]:
#define the 16 ROIs
sumrois={'Frontal': ['ctx.lh.caudalmiddlefrontal', 'ctx.rh.caudalmiddlefrontal', 'ctx.lh.lateralorbitofrontal', 'ctx.rh.lateralorbitofrontal', 'ctx.lh.medialorbitofrontal', 'ctx.rh.medialorbitofrontal', 'ctx.lh.parsopercularis', 'ctx.rh.parsopercularis', 'ctx.lh.parsorbitalis', 'ctx.rh.parsorbitalis', 'ctx.lh.parstriangularis', 'ctx.rh.parstriangularis', 'ctx.lh.precentral', 'ctx.rh.precentral', 'ctx.lh.rostralmiddlefrontal', 'ctx.rh.rostralmiddlefrontal', 'ctx.lh.superiorfrontal', 'ctx.rh.superiorfrontal', 'ctx-lh-caudalmiddlefrontal', 'ctx-rh-caudalmiddlefrontal', 'ctx-lh-lateralorbitofrontal', 'ctx-rh-lateralorbitofrontal', 'ctx-lh-medialorbitofrontal', 'ctx-rh-medialorbitofrontal', 'ctx-lh-parsopercularis', 'ctx-rh-parsopercularis', 'ctx-lh-parsorbitalis', 'ctx-rh-parsorbitalis', 'ctx-lh-parstriangularis', 'ctx-rh-parstriangularis', 'ctx-lh-precentral', 'ctx-rh-precentral', 'ctx-lh-rostralmiddlefrontal', 'ctx-rh-rostralmiddlefrontal', 'ctx-lh-superiorfrontal', 'ctx-rh-superiorfrontal'], 'Hippocampus': ['Left.Hippocampus', 'Right.Hippocampus', 'Left-Hippocampus', 'Right-Hippocampus'], 'Frontal_Parietal': ['ctx.lh.paracentral', 'ctx.rh.paracentral', 'ctx-lh-paracentral', 'ctx-rh-paracentral'], 'Parietal': ['ctx.lh.inferiorparietal', 'ctx.rh.inferiorparietal', 'ctx.lh.postcentral', 'ctx.rh.postcentral', 'ctx.lh.precuneus', 'ctx.rh.precuneus', 'ctx.lh.superiorparietal', 'ctx.rh.superiorparietal', 'ctx.lh.supramarginal', 'ctx.rh.supramarginal', 'ctx-lh-inferiorparietal', 'ctx-rh-inferiorparietal', 'ctx-lh-postcentral', 'ctx-rh-postcentral', 'ctx-lh-precuneus', 'ctx-rh-precuneus', 'ctx-lh-superiorparietal', 'ctx-rh-superiorparietal', 'ctx-lh-supramarginal', 'ctx-rh-supramarginal'], 'Insula': ['ctx.lh.insula', 'ctx.rh.insula', 'ctx-lh-insula', 'ctx-rh-insula'], 'Cingulate': ['ctx.lh.caudalanteriorcingulate', 'ctx.rh.caudalanteriorcingulate', 'ctx.lh.isthmuscingulate', 'ctx.rh.isthmuscingulate', 'ctx.lh.posteriorcingulate', 'ctx.rh.posteriorcingulate', 'ctx.lh.rostralanteriorcingulate', 'ctx.rh.rostralanteriorcingulate', 'ctx-lh-caudalanteriorcingulate', 'ctx-rh-caudalanteriorcingulate', 'ctx-lh-isthmuscingulate', 'ctx-rh-isthmuscingulate', 'ctx-lh-posteriorcingulate', 'ctx-rh-posteriorcingulate', 'ctx-lh-rostralanteriorcingulate', 'ctx-rh-rostralanteriorcingulate'], 'Occipital': ['ctx.lh.cuneus', 'ctx.rh.cuneus', 'ctx.lh.lateraloccipital', 'ctx.rh.lateraloccipital', 'ctx.lh.lingual', 'ctx.rh.lingual', 'ctx.lh.pericalcarine', 'ctx.rh.pericalcarine', 'ctx-lh-cuneus', 'ctx-rh-cuneus', 'ctx-lh-lateraloccipital', 'ctx-rh-lateraloccipital', 'ctx-lh-lingual', 'ctx-rh-lingual', 'ctx-lh-pericalcarine', 'ctx-rh-pericalcarine'], 'BasalGanglia': ['Left.Caudate', 'Right.Caudate', 'Left.Putamen', 'Right.Putamen', 'Left.Pallidum', 'Right.Pallidum', 'Left.Accumbens.area', 'Right.Accumbens.area', 'Left-Caudate', 'Right-Caudate', 'Left-Putamen', 'Right-Putamen', 'Left-Pallidum', 'Right-Pallidum', 'Left-Accumbens-area', 'Right-Accumbens-area'], 'Thalamus': ['Left.Thalamus.Proper', 'Right.Thalamus.Proper', 'Left-Thalamus-Proper', 'Right-Thalamus-Proper'], 'Cerebellum': ['Left.Cerebellum.White.Matter', 'Right.Cerebellum.White.Matter', 'Left.Cerebellum.Cortex', 'Right.Cerebellum.Cortex', 'Left-Cerebellum-White-Matter', 'Right-Cerebellum-White-Matter', 'Left-Cerebellum-Cortex', 'Right-Cerebellum-Cortex'], 'CorpusCallosum': ['CC_Posterior', 'CC_Mid_Posterior', 'CC_Central', 'CC_Mid_Anterior', 'CC_Anterior', 'CC_Posterior', 'CC_Mid_Posterior', 'CC_Central', 'CC_Mid_Anterior', 'CC_Anterior'], 'Ventricles': ['Left.Lateral.Ventricle', 'Right.Lateral.Ventricle', 'Left.Inf.Lat.Vent', 'Right.Inf.Lat.Vent', 'Left.choroid.plexus', 'Right.choroid.plexus', '3rd.Ventricle', '4th.Ventricle', 'CSF', 'Left-Lateral-Ventricle', 'Right-Lateral-Ventricle', 'Left-Inf-Lat-Vent', 'Right-Inf-Lat-Vent', 'Left-choroid-plexus', 'Right-choroid-plexus', '3rd-Ventricle', '4th-Ventricle', 'CSF'], 'VentralDC': ['Left.VentralDC', 'Right.VentralDC', 'Left-VentralDC', 'Right-VentralDC'], 'WM': ['Left.Cerebral.White.Matter', 'Right.Cerebral.White.Matter', 'Left-Cerebral-White-Matter', 'Right-Cerebral-White-Matter'], 'Brainstem': ['Brain.Stem', 'Brain-Stem'], 'Temporal': ['Left-Amygdala', 'Right-Amygdala', 'ctx-lh-entorhinal', 'ctx-lh-fusiform', 'ctx-lh-inferiortemporal', 'ctx-lh-middletemporal', 'ctx-lh-parahippocampal', 'ctx-lh-superiortemporal', 'ctx-lh-transversetemporal', 'ctx-rh-entorhinal', 'ctx-rh-fusiform', 'ctx-rh-inferiortemporal', 'ctx-rh-middletemporal', 'ctx-rh-parahippocampal', 'ctx-rh-superiortemporal', 'ctx-rh-transversetemporal', 'Left-Amygdala', 'Right-Amygdala', 'ctx-lh-entorhinal', 'ctx-lh-fusiform', 'ctx-lh-inferiortemporal', 'ctx-lh-middletemporal', 'ctx-lh-parahippocampal', 'ctx-lh-superiortemporal', 'ctx-lh-transversetemporal', 'ctx-rh-entorhinal', 'ctx-rh-fusiform', 'ctx-rh-inferiortemporal', 'ctx-rh-middletemporal', 'ctx-rh-parahippocampal', 'ctx-rh-superiortemporal', 'ctx-rh-transversetemporal']}
# Replace
for x, y in sumrois.items():
    sumrois[x] = [i.replace('.', '-') for i in y]

In [ ]:
#update the df
for sumroi,subrois in sumrois.items():
    df[sumroi] = df[subrois].sum(axis=1)
    df[sumroi] = df[sumroi].round(1)
    df.drop(subrois, axis=1, inplace=True)

In [ ]:
df["IDs"] = [i.split(".")[0] for i in df["IDs"]] # delete .reconall
df.drop(['BrainSegVol', 'BrainSegVolNotVent',
       'BrainSegVolNotVentSurf', 'SubCortGrayVol', 'MaskVol',
       'BrainSegVol-to-eTIV', 'MaskVol-to-eTIV'],inplace = True, axis = 1)

In [ ]:
#realtive vols 
for roi in df.columns[3:]:
    print(roi)
    df[f"{roi}_relative"] = df[roi]/df["eTIV"]

In [ ]:
#save the summed vols
df.to_csv(f"freesurfer_vol.csv",index=False)